In [3]:
from pathlib import Path
import os


from cleanair.scoot import load_model_and_metadata

In [ ]:
xpfp = "./experiments"  # root to filepaths directory
figure_dir = os.path.join(xpfp, "figures"))


Path(figure_dir.mkdir(exist_ok=True)

In [4]:
detector_id = "N00/002e1"
scoot_id = 0

model0, X_arr, Y_arr, metadata = load_model_and_metadata(detector_id,'10Feb_16Feb')

FileNotFoundError: [Errno 2] No such file or directory: '../data/models/N00_002e1/10Feb_16Feb/model.h5'

In [ ]:
plotly_results(sensor_df, detector_id, model0, X_arr[scoot_id], num_samples=1000)